In [1]:
# Data Handling
import pandas as pd
import os
pd.set_option('display.max_colwidth', None)

# Text Processing
import re
import unicodedata
from pathlib import Path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import numpy as np


In [2]:
out_dir = "project"
outputdirectory = Path(f"data_output/{out_dir}")

## Import Data

In [3]:
df =  pd.read_csv('data_input/dataset_final.csv')

In [4]:
df.head()

,text,page,chapter_name
0,"2©2009 Project Management Institute. Practice Standard for Project Risk Management1 CHAPTER 1 − INTRODUCTION\n 1.1 Purpose of the Practice Standard for Project Risk Management \n The purpose of the Practice Standard for Project Risk Management is to ( a ) provide a standard for project \nmanagement practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that \nare recognized as good practice on most projects most of the time and ( b ) provide a standard that is globally \napplicable and consistently applied. This practice standard has a descriptive purpose rather than one used for training or educational purposes. \n The Practice Standard for Project Risk Management covers risk management as it is applied to single \nprojects only. Like the PMBOK \n® Guide – Fourth Edition, this practice standard does not cover risk in programs \nor portfolios of projects. \n Chapter 11 of the PMBOK ® Guide – Fourth Edition, is the basis for the Practice Standard for Project Risk \nManagement . This practice standard is consistent with that chapter, emphasizing the concepts and principles relating to Project Risk Management. It is aligned with other PMI practice standards. \n Figure 1-1 compares the purposes of this practice standard to those of the PMBOK \n® Guide – Fourth Edition \nand textbooks, handbooks, and courses. \n Figure 1-1. Hierarchy of PMI Project Risk Management Resources",13,Introduction
1,"3©2009 Project Management Institute. Practice Standard for Project Risk Management1 CHAPTER 1 − INTRODUCTION\n This practice standard is organized in three main sections: \n 1. Introductory material including the framework, purpose, principles, context of, and introduction to \nProject Risk Management processes as deﬁ ned in the PMBOK ® Guide – Fourth Edition. \n 2. Principles underlying the six Project Risk Management processes in the PMBOK ® Guide – Fourth \nEdition. The six processes are as follows: \n Plan Risk Management, • \n Identify Risks, • \n Perform Qualitative Risk Analysis, • \n Perform Quantitative Risk Analysis, • \n Plan Risk Responses, and • \n Monitor and Control Risks. • \n Each of these six processes is described in a chapter that addresses the following four topics: \n( a ) purpose and objectives of the process; ( b ) critical success factors for the process; ( c ) tools and \ntechniques for the process; and ( d ) documenting the results of the process. \n 3. A glossary of terms which are used in this practice standard. \n This practice standard emphasizes those principles that are fundamental to effective, comprehensive, \nand successful Project Risk Management. These principles can and should be stated at a general level for \nseveral reasons: \n 1. Principles are expected to be agreed upon now and to be valid in the future. While tools and \ntechniques are constantly evolving, the principles have more stability and persistence. \n 2. Different projects, organizations, and situations will require different approaches to Project Risk \nManagement. In particular, risk management is a discipline that contains a series of processes to \napply to both large and small projects. Risk management will be more effective if its practice is \ntailored to the project and congruent with the organizational culture, processes and assets. There \nare many different ways of conducting risk management that may comply with the principles of Project Risk Management as presented in this practice standard. \n 3. The principles are applicable to projects carried out in a global context, reﬂ ecting the many \nbusiness and organizational arrangements between participants, for example, joint ventures \nbetween commercial and national companies, government and non-government organizations, and the cross-cultural environment often found on these project teams. \n The principles described herein can be used as a check for an organization’s processes. Practitioners can \nestablish processes

# Stocker chaque paragraphe comme un chunk distinct

In [5]:
# Fonction pour extraire les phrases du DataFrame et ajouter un chunk_id
def chunk_by_paragraph(df):
    chunks = []
    chunk_id = 1  # Initialiser le chunk_id
    
    for index, row in df.iterrows():
        text = row['text']
        page = row['page']
        chapter_name = row['chapter_name']
        
        # Séparation des paragraphes
        paragraphs = text.split("\n\n")
        
        for para in paragraphs:
            if para.strip():  # Assurez-vous que le paragraphe n'est pas vide
                chunks.append({
                    'chunk_id': chunk_id,   # Ajouter un chunk_id unique
                    'text': para,
                    'page': page,
                    'chapter_name': chapter_name
                })
                chunk_id += 1  # Incrémenter le chunk_id pour chaque paragraphe

    return pd.DataFrame(chunks)


In [6]:
# Charger le fichier CSV
df = pd.read_csv('data_input/dataset_final.csv')

# Extraire les phrases du DataFrame
sentences_df = chunk_by_paragraph(df)

# Afficher un aperçu des phrases extraites
sentences_df.head(100)

,chunk_id,text,page,chapter_name
0,1,"2©2009 Project Management Institute. Practice Standard for Project Risk Management1 CHAPTER 1 − INTRODUCTION\n 1.1 Purpose of the Practice Standard for Project Risk Management \n The purpose of the Practice Standard for Project Risk Management is to ( a ) provide a standard for project \nmanagement practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that \nare recognized as good practice on most projects most of the time and ( b ) provide a standard that is globally \napplicable and consistently applied. This practice standard has a descriptive purpose rather than one used for training or educational purposes. \n The Practice Standard for Project Risk Management covers risk management as it is applied to single \nprojects only. Like the PMBOK \n® Guide – Fourth Edition, this practice standard does not cover risk in programs \nor portfolios of projects. \n Chapter 11 of the PMBOK ® Guide – Fourth Edition, is the basis for the Practice Standard for Project Risk \nManagement . This practice standard is consistent with that chapter, emphasizing the concepts and principles relating to Project Risk Management. It is aligned with other PMI practice standards. \n Figure 1-1 compares the purposes of this practice standard to those of the PMBOK \n® Guide – Fourth Edition \nand textbooks, handbooks, and courses. \n Figure 1-1. Hierarchy of PMI Project Risk Management Resources",13,Introduction
1,2,"3©2009 Project Management Institute. Practice Standard for Project Risk Management1 CHAPTER 1 − INTRODUCTION\n This practice standard is organized in three main sections: \n 1. Introductory material including the framework, purpose, principles, context of, and introduction to \nProject Risk Management processes as deﬁ ned in the PMBOK ® Guide – Fourth Edition. \n 2. Principles underlying the six Project Risk Management processes in the PMBOK ® Guide – Fourth \nEdition. The six processes are as follows: \n Plan Risk Management, • \n Identify Risks, • \n Perform Qualitative Risk Analysis, • \n Perform Quantitative Risk Analysis, • \n Plan Risk Responses, and • \n Monitor and Control Risks. • \n Each of these six processes is described in a chapter that addresses the following four topics: \n( a ) purpose and objectives of the process; ( b ) critical success factors for the process; ( c ) tools and \ntechniques for the process; and ( d ) documenting the results of the process. \n 3. A glossary of terms which are used in this practice standard. \n This practice standard emphasizes those principles that are fundamental to effective, comprehensive, \nand successful Project Risk Management. These principles can and should be stated at a general level for \nseveral reasons: \n 1. Principles are expected to be agreed upon now and to be valid in the future. While tools and \ntechniques are constantly evolving, the principles have more stability and persistence. \n 2. Different projects, organizations, and situations will require different approaches to Project Risk \nManagement. In particular, risk management is a discipline that contains a series of processes to \napply to both large and small projects. Risk management will be more effective if its practice is \ntailored to the project and congruent with the organizational culture, processes and assets. There \nare many different ways of conducting risk management that may comply with the principles of Project Risk Management as presented in this practice standard. \n 3. The principles are applicable to projects carried out in a global context, reﬂ ecting the many \nbusiness and organizational arrangements between participants, for example, joint ventures \nbetween commercial and national companies, government and non-government organizations, and the cross-cultural environment often found on these project teams. \n The principles described herein can be used as a check for an organization’s processes. Practitioners can \nestabl

## Clean data 

In [7]:
# Load the default English stopwords
stop_words = set(stopwords.words('english'))
# Define your custom stop words
custom_stop_words = {
    # Articles
    'a', 'an', 'the',

    # Prepositions
    'in', 'on', 'at', 'by', 'for', 'with', 'of',

    # Conjunctions
    'and', 'or', 'but', 'if',

    # Pronouns
    'i', 'you', 'he', 'she', 'it', 'we', 'they', 'this', 'that',

    # Common Verbs
    'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'do', 'does',

    # Adverbs
    'very', 'so', 'just',

    # Quantifiers
    'all', 'some', 'many', 'most',

    # Additional Words
    'other', 'chapter', 'appendix','PMI'

    # Single Letters
    'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',

    # Specific Words and Numbers
    '2009', 'to', 'can', 'may', 'of','.'
}

# Update the NLTK stopwords with the custom stopwords
stop_words.update(custom_stop_words)

In [8]:
def clean_text(text):
    # Normalize unicode characters (remove accents)
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove ellipses (sequences of three or more periods)
    text = re.sub(r'\.\.\.+', ' ', text)
    
    # Remove digits
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation and special characters except spaces
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    
    # Join words back to a single string
    cleaned_text = ' '.join(words)
    
    # Remove multiple spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # Strip leading and trailing spaces
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text


In [9]:
sentences_df['cleaned_text'] = sentences_df['text'].apply(lambda x : clean_text(x))

In [10]:
sentences_df.head()

,chunk_id,text,page,chapter_name,cleaned_text
0,1,"2©2009 Project Management Institute. Practice Standard for Project Risk Management1 CHAPTER 1 − INTRODUCTION\n 1.1 Purpose of the Practice Standard for Project Risk Management \n The purpose of the Practice Standard for Project Risk Management is to ( a ) provide a standard for project \nmanagement practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that \nare recognized as good practice on most projects most of the time and ( b ) provide a standard that is globally \napplicable and consistently applied. This practice standard has a descriptive purpose rather than one used for training or educational purposes. \n The Practice Standard for Project Risk Management covers risk management as it is applied to single \nprojects only. Like the PMBOK \n® Guide – Fourth Edition, this practice standard does not cover risk in programs \nor portfolios of projects. \n Chapter 11 of the PMBOK ® Guide – Fourth Edition, is the basis for the Practice Standard for Project Risk \nManagement . This practice standard is consistent with that chapter, emphasizing the concepts and principles relating to Project Risk Management. It is aligned with other PMI practice standards. \n Figure 1-1 compares the purposes of this practice standard to those of the PMBOK \n® Guide – Fourth Edition \nand textbooks, handbooks, and courses. \n Figure 1-1. Hierarchy of PMI Project Risk Management Resources",13,Introduction,project management institute practice standard project risk management introduction purpose practice standard project risk management purpose practice standard project risk management provide standard project management practitioners stakeholders defi nes aspects project risk management recognized good practice projects time b provide standard globally applicable consistently applied practice standard descriptive purpose rather one used training educational purposes practice standard project risk management covers risk management applied single projects like pmbok guide fourth edition practice standard cover risk programs portfolios projects pmbok guide fourth edition basis practice standard project risk management practice standard consistent emphasizing concepts principles relating project risk management aligned pmi practice standards figure compares purposes practice standard pmbok guide fourth edition textbooks handbooks courses figure hierarchy pmi project risk management resources
1,2,"3©2009 Project Management Institute. Practice Standard for Project Risk Management1 CHAPTER 1 − INTRODUCTION\n This practice standard is organized in three main sections: \n 1. Introductory material including the framework, purpose, principles, context of, and introduction to \nProject Risk Management processes as deﬁ ned in the PMBOK ® Guide – Fourth Edition. \n 2. Principles underlying the six Project Risk Management processes in the PMBOK ® Guide – Fourth \nEdition. The six processes are as follows: \n Plan Risk Management, • \n Identify Risks, • \n Perform Qualitative Risk Analysis, • \n Perform Quantitative Risk Analysis, • \n Plan Risk Responses, and • \n Monitor and Control Risks. • \n Each of these six processes is described in a chapter that addresses the following four topics: \n( a ) purpose and objectives of the process; ( b ) critical success factors for the process; ( c ) tools and \ntechniques for the process; and ( d ) documenting the results of the process. \n 3. A glossary of terms which are used in this practice standard. \n This practice standard emphasizes those principles that are fundamental to effective, comprehensive, \nand successful Project Risk Management. These principles can and should be stated at a general level for \nseveral reasons: \n 1. Principles are expected to be agreed upon now and to be valid in the future. While tools and \ntechniques are constantly evolving, the principles have more stability and persistence. \n 2. Different projec

In [11]:
# Filtrer le DataFrame pour ne garder que les paragraphes d'un chapitre spécifique
def filter_by_chapter(df, chapter_name):
    return df[df['chapter_name'] == chapter_name]

In [12]:
chapter_name = "Introduction" 
chapter_df = filter_by_chapter(sentences_df, chapter_name)

In [13]:
chapter_df.shape

(6, 5)

In [14]:
chapter_df.isna().sum()

chunk_id        0
text            0
page            0
chapter_name    0
cleaned_text    0
dtype: int64

In [15]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

In [ ]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(chapter_df, model='llama3:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

Here is the extracted ontology of terms mentioned in the given context:

[
   {
       "node_1": "Practice Standard",
       "node_2": "Project Risk Management",
       "edge": "describes"
   },
   {
       "node_1": "Introduction",
       "node_2": "Practice Standard",
       "edge": "introduces"
   },
   {
       "node_1": "Purpose",
       "node_2": "Practice Standard",
       "edge": "defines"
   },
   {
       "node_1": "Project Practitioners",
       "node_2": "Stakeholders",
       "edge": "define"
   },
   {
       "node_1": "Aspects",
       "node_2": "Project Risk Management",
       "edge": "covers"
   },
   {
       "node_1": "Recognized Good Practice",
       "node_2": "Projects",
       "edge": "applies"
   },
   {
       "node_1": "Time",
       "node_2": "Provide",
       "edge": "defines"
   },
   {
       "node_1": "Globally Applicable",
       "node_2": "Consistently Applied",
       "edge": "emphasizes"
   },
   {
       "node_1": "Descriptive Purpose",
       "node